Importing Libraries needed to run script

In [65]:
import os
import numpy as np
import pandas as pd1
import shutil
import csv

In [66]:
models = os.listdir("Modules") #modules list as array 
print(models)

['top_alu8bit.v', 'top_knowels_32bit.v', 'top_koggestone_32bit.v', 'top_lander_fisher_32bit.v', 'top_master_slave.v', 'top_mealy.v', 'top_moore.v', 'top_ripple4bit.v']


In [67]:
top_modules = []
for mod in models:
    if (mod[:3]=='top'):
        top_modules.append(mod[:-2])
    print(mod)
print(top_modules)


top_alu8bit.v
top_knowels_32bit.v
top_koggestone_32bit.v
top_lander_fisher_32bit.v
top_master_slave.v
top_mealy.v
top_moore.v
top_ripple4bit.v
['top_alu8bit', 'top_knowels_32bit', 'top_koggestone_32bit', 'top_lander_fisher_32bit', 'top_master_slave', 'top_mealy', 'top_moore', 'top_ripple4bit']


top_master_slave.v


In [68]:
os.system("vivado -mode batch -source tcl_create.tcl")   #creating new project

0

In [69]:
for mod in models:
    os.system("vivado -mode batch -source  tcl_add.tcl -tclargs {}".format(mod))

In [70]:
os.mkdir("results")

In [71]:
for filename in models:
    os.mkdir("results/"+filename[:-2])
    os.system("vivado -mode batch -source  tcl_run.tcl -tclargs {}".format(filename))
    print("{} reported".format(filename)) 

top_alu8bit.v reported
top_knowels_32bit.v reported
top_koggestone_32bit.v reported
top_lander_fisher_32bit.v reported
top_master_slave.v reported
top_mealy.v reported
top_moore.v reported
top_ripple4bit.v reported


Generating Result.csv file 
The approach is to search for certain text in the generated reports and extract out the numerical values needed.

In [72]:
data_files = os.listdir("results")  
print(data_files)

['top_alu8bit', 'top_knowels_32bit', 'top_koggestone_32bit', 'top_lander_fisher_32bit', 'top_master_slave', 'top_mealy', 'top_moore', 'top_ripple4bit']


In [87]:
file_types = ['power.txt','timing.txt','utilization.txt']

req_data = ['Total On-Chip Power','Data Path Delay','Slice LUTs']

Power = []
Delay = []
LUTs = []

In [88]:

for data in data_files :
    for file in file_types:
        with open("results/{}/{}".format(data,file),'r') as File:
             content = File.read()

        words = content.split('\n')   

        for req_word in words:
            if file_types[0] in file:
                if req_data[0] in req_word:
                    pwr = float(req_word.split()[6])
                    print(pwr)
                    Power.append(pwr)
  

        for req_word in words:
            if file_types[1] in file:
                if req_data[1] in req_word:
                    delay = float(req_word.split()[3][:-2])
                    print(delay)
                    Delay.append(delay)


        for req_word in words:
            if file_types[2] in file:
                if req_data[2] in req_word:
                    luts = float(req_word.split()[4])
                    print(luts) 
                    LUTs.append(luts) 
                    print("_____")          

             




11.055
13.83
131.0
_____
25.791
18.331
133.0
_____
25.741
18.177
131.0
_____
23.749
17.48
73.0
_____
0.639
4.767
1.0
_____
0.122
0.0
_____
0.529
4.813
5.0
_____
0.122
0.0
_____


In [91]:
Final_results = [Power,Delay,LUTs]
row_name = ['Power','Delay','LUTs']

df = pd1.DataFrame(Final_results, index = row_name)
df.columns = data_files

df.to_csv('Final_results.csv', index = row_name)
print(df)

       top_alu8bit  top_knowels_32bit  top_koggestone_32bit  \
Power       11.055             25.791                25.741   
Delay       13.830             18.331                18.177   
LUTs       131.000            133.000               131.000   

       top_lander_fisher_32bit  top_master_slave  top_mealy  top_moore  \
Power                   23.749             0.639      0.122      0.529   
Delay                   17.480             4.767      4.813        NaN   
LUTs                    73.000             1.000      0.000      5.000   

       top_ripple4bit  
Power           0.122  
Delay             NaN  
LUTs            0.000  
